In [ ]:
import os
import fsspec
import pandas as pd
os.getcwd()

In [ ]:
file_list = os.listdir('data')
print(file_list)

In [ ]:
fs_write = fsspec.filesystem(
    's3',
    profile='osn-renc',  ## This is the profile name you configured above.
    client_kwargs={'endpoint_url': 'https://renc.osn.xsede.org'}
)

In [ ]:
fs_write.ls('rsignellbucket2/nwc/nhm_prms_v1_1_gridmet/raw_data')

In [ ]:
complete_df = pd.DataFrame(columns=['huc12', 'date'])
for f in file_list:
    var_name = f.split('_mo')[0].split('huc12_')[1]
    print(var_name)
    df = pd.read_csv(os.path.join('data', f), dtype={'huc12': 'string'})
    #print(df.head())
    long_df = pd.melt(df, id_vars='huc12', var_name='date', value_name=var_name)
    complete_df = complete_df.merge(long_df, on=['huc12', 'date'], how='right')
    #print(complete_df.head())
    fname=f'rsignellbucket2/nwc/nhm_prms_v1_1_gridmet/raw_data/{f}'
    print(f'writing {fname}')
    outfile = fs_write.open(fname, mode='w')
    print(df.head())
    df.to_csv(outfile, index=False)
    # fs_write.upload('nhm_panel_viz.ipynb', 'rsignellbucket2/nwc/nhm_panel_viz.ipynb')
complete_df['date'] =  pd.to_datetime(complete_df['date'], format='%Y-%m')
complete_fname='rsignellbucket2/nwc/nhm_prms_v1_1_gridmet/nhm_prms_v1_1_gridmet.csv'
print(f'writing {complete_fname}')
#complete_df['huc12'] = complete_df["huc12"].map(str)
complete_outfile = fs_write.open(complete_fname, mode='w')
complete_df.to_csv(complete_outfile, index=False)

In [ ]:
fs_write.ls('rsignellbucket2/nwc/nhm_prms_v1_1_gridmet/')

In [ ]:
fs_write.ls('rsignellbucket2/nwc/nhm_prms_v1_1_gridmet/')